In [1]:
import sklearn as skl

In [7]:
import urllib3
from math import sqrt, fabs, exp
import matplotlib.pyplot as plot
import numpy

In [24]:
filepath = "C:/Users/28chr/Downloads/glass.txt"
data = open(filepath, mode='r')

In [25]:
xList = []
for line in data:
    row = str(line).strip().split(",")
    xList.append(row)
print(xList[0])

['1', '1.52101', '13.64', '4.49', '1.10', '71.78', '0.06', '8.75', '0.00', '0.00', '1']


In [27]:
import sys
from math import sqrt, fabs, exp
import matplotlib.pyplot as plot
def S(z,gamma):
    if gamma >= fabs(z):
        return 0.0
    if z > 0.0:
        return z - gamma
    else:
        return z + gamma
def Pr(b0,b,x):
    n = len(x)
    sum = b0
    for i in range(n):
        sum += b[i]*x[i]
        if sum < -100: 
            sum = -100
            return 1.0/(1.0 + exp(-sum))

In [42]:
filepath = "C:/Users/28chr/Downloads/sonar.txt"
data = open(filepath, mode='r')

In [43]:
xList = []
for line in data:
    row = str(line).strip().split(",")
    xList.append(row)

In [44]:
xNum = []
labels = []
print(xList[0])
for row in xList:
    lastCol = row.pop()
    if lastCol == "M":
        labels.append(1.0)
    else:
        labels.append(0.0)
    attrRow = [float(elt) for elt in row]
    xNum.append(attrRow)

['0.0200', '0.0371', '0.0428', '0.0207', '0.0954', '0.0986', '0.1539', '0.1601', '0.3109', '0.2111', '0.1609', '0.1582', '0.2238', '0.0645', '0.0660', '0.2273', '0.3100', '0.2999', '0.5078', '0.4797', '0.5783', '0.5071', '0.4328', '0.5550', '0.6711', '0.6415', '0.7104', '0.8080', '0.6791', '0.3857', '0.1307', '0.2604', '0.5121', '0.7547', '0.8537', '0.8507', '0.6692', '0.6097', '0.4943', '0.2744', '0.0510', '0.2834', '0.2825', '0.4256', '0.2641', '0.1386', '0.1051', '0.1343', '0.0383', '0.0324', '0.0232', '0.0027', '0.0065', '0.0159', '0.0072', '0.0167', '0.0180', '0.0084', '0.0090', '0.0032', 'R']


In [45]:
nrow = len(xNum)
ncol = len(xNum[1])
alpha = 0.8

In [46]:
xMeans = []
xSD = []
for i in range(ncol):
    col = [xNum[j][i] for j in range(nrow)]
    mean = sum(col)/nrow
    xMeans.append(mean)
    colDiff = [(xNum[j][i] - mean) for j in range(nrow)]
    sumSq = sum([colDiff[i] * colDiff[i] for i in range(nrow)])
    stdDev = sqrt(sumSq/nrow)
    xSD.append(stdDev)

In [47]:
xNormalized = []
for i in range(nrow):
    rowNormalized = [(xNum[i][j] - xMeans[j])/xSD[j] for j in range(ncol)]
    xNormalized.append(rowNormalized)

In [48]:
meanLabel = sum(labels)/nrow
sdLabel = sqrt(sum([(labels[i] - meanLabel) * (labels[i] - meanLabel) for i in range(nrow)])/nrow)

In [49]:
sumWxr = [0.0] * ncol
sumWxx = [0.0] * ncol
sumWr = 0.0
sumW = 0.0

In [50]:
for iRow in range(nrow):
    p = meanLabel
    w = p * (1.0 - p)
    #residual for logistic
    r = (labels[iRow] - p) / w
    x = xNormalized[iRow]
    sumWxr = [sumWxr[i] + w * x[i] * r for i in range(ncol)]
    sumWxx = [sumWxx[i] + w * x[i] * x[i] for i in range(ncol)]
    sumWr = sumWr + w * r
    sumW = sumW + w

In [51]:
avgWxr = [sumWxr[i]/nrow for i in range(ncol)]
avgWxx = [sumWxx[i]/nrow for i in range(ncol)]
maxWxr = 0.0
for i in range(ncol):
    val = abs(avgWxr[i])
    if val > maxWxr:
        maxWxr = val

In [52]:
#calculate starting value for lambda
lam = maxWxr/alpha
#this value of lambda corresponds to beta = list of 0's
#initialize a vector of coefficients beta
beta = [0.0] * ncol
beta0 = sumWr/sumW
#initialize matrix of betas at each step
betaMat = []
betaMat.append(list(beta))
beta0List = []
beta0List.append(beta0)

In [53]:
nSteps = 100
lamMult = 0.93 #100 steps gives reduction by factor of 1000 in lambda
#(recommended by authors)
nzList = []

In [54]:
for iStep in range(nSteps):
    #decrease lambda
    lam = lam * lamMult

    betaIRLS = list(beta)
    beta0IRLS = beta0
    distIRLS = 100.0
    #Middle loop to calculate new betas with fixed IRLS weights and
    #probabilities
    iterIRLS = 0
        while distIRLS > 0.01:
        iterIRLS += 1
        iterInner = 0.0
        betaInner = list(betaIRLS)
        beta0Inner = beta0IRLS
        distInner = 100.0
        while distInner > 0.01:
            iterInner += 1
            if iterInner > 100: break
            betaStart = list(betaInner)
            for iCol in range(ncol):
                sumWxr = 0.0
                sumWxx = 0.0
                sumWr = 0.0
                sumW = 0.0
                for iRow in range(nrow):
                    x = list(xNormalized[iRow])
                    y = labels[iRow]
                    p = Pr(beta0IRLS, betaIRLS, x)
                    if abs(p) < 1e-5:
                        p = 0.0
                        w = 1e-5
                    elif abs(1.0 - p) < 1e-5:
                        p = 1.0
                        w = 1e-5
                    else:
                        w = p * (1.0 - p)
                    z = (y - p) / w + beta0IRLS + sum([x[i] *
                    betaIRLS[i] for i in range(ncol)])
                    r = z - beta0Inner - sum([x[i] * betaInner[i] for i in range(ncol)])
                    sumWxr += w * x[iCol] * r
                    sumWxx += w * x[iCol] * x[iCol]
                    sumWr += w * r
                    sumW += w
                avgWxr = sumWxr / nrow
                avgWxx = sumWxx / nrow
            beta0Inner = beta0Inner + sumWr / sumW
            uncBeta = avgWxr + avgWxx * betaInner[iCol]
            betaInner[iCol] = S(uncBeta, lam * alpha) / (avgWxx + lam * (1.0 - alpha))
            sumDiff = sum([abs(betaInner[n] - betaStart[n]) for n in range(ncol)])
            sumBeta = sum([abs(betaInner[n]) for n in range(ncol)])
            distInner = sumDiff/sumBeta
        a = sum([abs(betaIRLS[i] - betaInner[i]) for i in range(ncol)])
        b = sum([abs(betaIRLS[i]) for i in range(ncol)])
        distIRLS = a / (b + 0.0001)
        dBeta = [betaInner[i] - betaIRLS[i] for i in range(ncol)]
        gradStep = 1.0
        temp = [betaIRLS[i] + gradStep * dBeta[i] for i in range(ncol)]
        betaIRLS = list(temp)
    beta = list(betaIRLS)
    beta0 = beta0IRLS
    betaMat.append(list(beta))
    beta0List.append(beta0)
    nzBeta = [index for index in range(ncol) if beta[index] != 0.0]
    for q in nzBeta:
        if not(q in nzList):
            nzList.append(q)
            

            
            
            

IndentationError: expected an indented block after 'while' statement on line 11 (1218664036.py, line 12)

In [55]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [56]:
X = np.array([[1,1], [1,2], [2,2], [2,3]])
y = np.dot(X, np.array([1,2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)

1.0

In [57]:
reg.coef_

array([1., 2.])

In [58]:
reg.intercept_

3.0

In [59]:
reg.predict(np.array([[3, 5]]))

array([16.])

In [77]:
filepath = "C:/Users/28chr/Downloads/sonar.txt"
file = open(filepath, "r")

In [78]:
X = []
y = []
i = 0
for line in file:
    if i % 5 != 4:
        x = line.strip().split(",")
        for i in range(len(x) - 1):
            x[i] = float(x[i])
        lastX = x.pop()
        if lastX == 'R':
            y.append(0)
        else:
            y.append(1)
        X.append(x)
    i = i + 1
print(X)
print(y)

[[0.02, 0.0371, 0.0428, 0.0207, 0.0954, 0.0986, 0.1539, 0.1601, 0.3109, 0.2111, 0.1609, 0.1582, 0.2238, 0.0645, 0.066, 0.2273, 0.31, 0.2999, 0.5078, 0.4797, 0.5783, 0.5071, 0.4328, 0.555, 0.6711, 0.6415, 0.7104, 0.808, 0.6791, 0.3857, 0.1307, 0.2604, 0.5121, 0.7547, 0.8537, 0.8507, 0.6692, 0.6097, 0.4943, 0.2744, 0.051, 0.2834, 0.2825, 0.4256, 0.2641, 0.1386, 0.1051, 0.1343, 0.0383, 0.0324, 0.0232, 0.0027, 0.0065, 0.0159, 0.0072, 0.0167, 0.018, 0.0084, 0.009, 0.0032], [0.0453, 0.0523, 0.0843, 0.0689, 0.1183, 0.2583, 0.2156, 0.3481, 0.3337, 0.2872, 0.4918, 0.6552, 0.6919, 0.7797, 0.7464, 0.9444, 1.0, 0.8874, 0.8024, 0.7818, 0.5212, 0.4052, 0.3957, 0.3914, 0.325, 0.32, 0.3271, 0.2767, 0.4423, 0.2028, 0.3788, 0.2947, 0.1984, 0.2341, 0.1306, 0.4182, 0.3835, 0.1057, 0.184, 0.197, 0.1674, 0.0583, 0.1401, 0.1628, 0.0621, 0.0203, 0.053, 0.0742, 0.0409, 0.0061, 0.0125, 0.0084, 0.0089, 0.0048, 0.0094, 0.0191, 0.014, 0.0049, 0.0052, 0.0044], [0.0262, 0.0582, 0.1099, 0.1083, 0.0974, 0.228, 0.2431,

In [79]:
X = np.array(X)
y = np.array(y)
reg = LinearRegression().fit(X, y)

In [82]:
res = []
file2 = open(filepath, "r")
i = 0
for line in file2:
    if i % 5 == 4:
        x = line.strip().split(",")
        for i in range(len(x) - 1):
            x[i] = float(x[i])
        lastX = x.pop()
        y = -1
        if lastX == 'R':
            y = 0
        else:
            y = 1
        out = reg.predict(np.array([x]))
        res.append((out, y))
    i = i + 1

In [92]:
mis = 0
n = 0
for tup in res:
    if tup[0][0] > 0.5 and tup[1] == 0:
        mis += 1
    elif tup[0][0] < 0.5 and tup[1] == 1:
        mis += 1
    n += 1
print(mis)
print(mis / n)
print(n)

3
0.07317073170731707
41


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sklearn.metrics (from versions: none)
ERROR: No matching distribution found for sklearn.metrics
